# 1. Libraries

In [ ]:
# Aca se importan las librerias que vas a usar
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from sklearn import metrics
from scipy import stats
#from utils.Validator import *
import pickle
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 2. Data Collection

In [ ]:
# Aca leemos los datasets de entranamiento y test (creados en el preprocessing)
train = pd.read_csv(r"../data/modeling/03_train.csv",sep="|", dtype={'codunicocli_limpio':'str',})   
test = pd.read_csv(r"../data/modeling/03_test.csv",sep="|", dtype={'codunicocli_limpio':'str',}) 


Train

In [ ]:
#Escalón1
#Por definición conceptual se excluye los clientes con montos de remesas acumuladas por debajo de $200
df00=train[train['MONTO_TOTAL']>=200]

In [ ]:
#Escalón3
#Idenfica alertas en el 3er escalón - Directo a evaluación
df0=train[(train['CTD_TRX']>=25) | (train['MONTO_TOTAL']>=20000)]
df0["IF_LABEL"]=3

In [ ]:
#Escalón2
# Identifica a los clientes para la segmentación
df1=df00[(df00['CTD_TRX']<25) & (df00['MONTO_TOTAL']<20000)]
df2 = df1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'FLG_PERFIL']]


Test

In [ ]:
#Escalón1
#Por definición conceptual se excluye los clientes con montos de remesas acumuladas por debajo de $200
df_00=test[test['MONTO_TOTAL']>=200]

In [ ]:
#Escalón3
#Idenfica alertas en el 3er escalón - Directo a evaluación
df_0=test[(test['CTD_TRX']>=25) | (test['MONTO_TOTAL']>=20000)]
df_0["IF_LABEL"]=3

In [ ]:
#Escalón2
# Identifica a los clientes para la segmentación
df_1=df_00[(df_00['CTD_TRX']<25) & (df_00['MONTO_TOTAL']<20000)]
df_2 = df_1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'FLG_PERFIL']]

# 3. Escalamiento de variables

In [ ]:
# Aca escalamos las variables en caso sea necesario

#Train
scaler = MinMaxScaler()
df2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'FLG_PERFIL_SC']] = scaler.fit_transform(df2)
df2=df2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'FLG_PERFIL_SC']]
df2

#Test
df_2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'FLG_PERFIL_SC']] = scaler.fit_transform(df_2)
df_2=df_2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'FLG_PERFIL_SC']]
df_2

# 4. Model

In [ ]:
# Aca entrenamos el modelo final obtenido del noteebok "modeling"


# 5. Export .model

In [ ]:
# Aca exportamos el .model en la carpeta ../src/02_models/XXXXX.model

model = pickle.load(open(r"../src/02_models/IF.model", 'rb'))


# 5. Export data

In [ ]:
# Aca exportamos el resultado de la inferencia del train y test en 
# - ../data/modeling/04_predicted_train.csv
# - ../data/modeling/04_predicted_test.csv

#Train
df1['IF_LABEL']=model.predict(df2)
#Alertas según el modelo Isolations 
df3=df1[df1["IF_LABEL"]==-1]
#Unimos las alertas del escalón 3 y 2
alertas_train=pd.concat([df0, df3], ignore_index=True)
alertas_train.to_csv(r"../data/modeling/04_predicted_train.csv",index=False,sep="|")



In [ ]:
alertas_train

In [ ]:
#Test
df_1['IF_LABEL'] = model.predict(df_2)

In [ ]:
#Test

#Alertas según el modelo Isolations 
df_3=df_1[df_1["IF_LABEL"]==-1]
#Unimos las alertas del escalón 3 y 2
alertas_test=pd.concat([df_0, df_3], ignore_index=True)
alertas_test.to_csv(r"../data/modeling/04_predicted_test.csv",index=False,sep="|")

In [ ]:
alertas_test